In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hypothesis as h

import time
plt.rcParams['figure.figsize'] = 10, 5

# Analysis of the rank of the constrain matrix

If not stated otherwise, we assume that there is at most one measurement per time instance.

In this notebook we use a little rearanged definitions of the left and right parts of matrix:

Let us write $\pmb{f_n}^\top$ as $[1\ \pmb{g_n}^\top]^\top$, and similarly
$\tilde{\pmb{a}}_m^\top = [\pmb{a_m}^\top 1]^\top$. We can then 
write the whole constraint vector as:
$$\begin{bmatrix}
\text{vect}(\tilde{\pmb{a}}_m\pmb{f}_n^\top)^\top & \text{vect}( 
\pmb{g_n} \pmb{f_n}^\top)^\top
\end{bmatrix}$$
Then we have the needed constrains on the left, and constrains added due to the 
regularisation on the right.
From the Lemmas from the paper we know that we can write:
$$\text{vect}(\pmb{f_n f_n^\top}) = \pmb{R} \pmb{f_n^e},$$
where $\pmb{R}$ is some (sparse) matrix, and $\pmb{f_n^e}$ is a vector 
similar in structure to $\pmb{f_n}$, and can be written as a concatenation of  $\pmb{f_n}$ and $\pmb{f}_n^{r}$.
This means we can write our full system of equations as:
\begin{align}
b &=
\begin{bmatrix}
\text{vect}(\pmb{a_m f_n}^\top)^\top & (\pmb{f_n^e})^\top
\end{bmatrix}
\begin{bmatrix}
\pmb{C} \\
\pmb{R^\top L} \\\end{bmatrix}\\
\end{align}
Or using $\tilde{\pmb{a}}_m$:
\begin{align}
b &=
\begin{bmatrix}
\text{vect}(\tilde{\pmb{a}}_m \pmb{f}_n^\top)^\top & \pmb{f}_n^{r\top}
\end{bmatrix}
\begin{bmatrix}
\pmb{C} \\
\pmb{R}^\top \pmb{L} \\\end{bmatrix}
\end{align}

And so for us left hand side refers to $\text{vect}(\tilde{\pmb{a}}_m \pmb{f}_n^\top)^\top$ and right hand side refers to $\pmb{f}_n^{r\top}$.


## Rank vs number of measurements: simulations
In those experiments, the number of dimentions $D$, number of constrains $K$ and number of positions $N$ is fixed,
and for several different ranks the number of measurements is increased. The second experiment allows for many measurements per time instance. We can see that with at most one measurement per time we have better results for more than D+1 anchors, but for D+1 anchors there many measurements setup is better.

In [ ]:
plt.rcParams['figure.figsize'] = 10, 5

experiment_params={
    "n_dimensions": 2,
    "n_constraints": 5,
    "n_positions": 60,
    "n_repetitions": 100,
    "full_matrix": True,
    "n_anchors_list": [1, 2, 4, 8, 100],
    "one_per_time": True,
}

ranks, params = h.matrix_rank_experiment(experiment_params)

h.plot_results(ranks, params)
plt.show()

experiment_params={
    "n_dimensions": 2,
    "n_constraints": 5,
    "n_positions": 20,
    "n_repetitions": 100,
    "full_matrix": True,
    "n_anchors_list": [1, 2, 4, 8, 100],
    "one_per_time": False,
}

ranks, params = h.matrix_rank_experiment(experiment_params)

h.plot_results(ranks, params)
plt.show()

## Rank vs number of measurements: theory and simulations
First plots in this part focus on the rank of the left hand side of the matrix.

In [ ]:
n_constrains = 5
experiment_params={
    "n_dimensions": 2,
    "n_constraints": n_constrains,
    "n_repetitions": 1000,
    "full_matrix": True,
    "n_anchors_list": [1, 2, 3, 10],
    "n_positions": 40,
    "one_per_time": True,
}

start = time.time()
ranks, params = h.matrix_rank_experiment(experiment_params)
end = time.time()
print("elapsed time: {:.2f}s".format(end - start))

In [ ]:
n_measurement_list = params["second_list"]

probabilities = []
for idx, n_anchors in enumerate(params["n_anchors_list"]):
    print("{} anchors".format(n_anchors))
    probabilities.append([h.probability_upper_bound_any_measurements(
        params["n_dimensions"],
        params["n_constraints"],
        n_measurements=n,
        position_wise=False,
        n_anchors=n_anchors,
        n_positions=np.Infinity
    ) for n in n_measurement_list])
probabilities = np.array(probabilities)


### Plot and save results

In [ ]:
import pickle
from plotting_tools import make_dirs_safe
directory = 'theory/'
make_dirs_safe(directory)
name = 'first_one_per_time_plots'
data = {"ranks": ranks, "probabilities":probabilities, "params": params}
pickle.dump(data, open( directory + name + '.p', "wb" ) )

plt.rcParams['figure.figsize'] = 10, 5
f, ax = plt.subplots()
for idx, n_anchors in enumerate(params["n_anchors_list"]):

    max_rank = params["max_rank"]
    mean =  np.mean(ranks[:, idx, :] >= max_rank, axis=-1)
    std = np.std(ranks[:, idx, :] >= max_rank, axis=-1)
    anchor_condition = np.mean(params["anchor_condition"][:, idx, :], axis=-1)
    frame_condition = np.mean(params["frame_condition"][:, idx, :], axis=-1)
    both_conditions = np.mean((params["anchor_condition"] * params["frame_condition"])[:, idx, :], axis=-1)


    p = plt.step(
        n_measurement_list,
        probabilities[idx],
        label="calculated prob. {} anchors".format(n_anchors), # using the old model, what needs to be changed?
        linestyle=":",
        where='post')
    plt.step(
        n_measurement_list,
        mean,
        label="estimated prob. {} anchors".format(n_anchors),
        color=p[-1].get_color(),
        where='post')

plt.xlabel("number of measurements")
plt.ylim(0)
plt.grid()
plt.legend()
matrix_name = "Full matrix" if params["full_matrix"] else "Left hand side"
plt.title(matrix_name + ", one measurement at time, complexity {}".format(params["n_constraints"]))
plt.savefig(directory + name + str(params["full_matrix"]) + '.pdf')
plt.show()